In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from tqdm import tqdm
from sklearn.model_selection import train_test_split
import tensorflow as tf
from keras.models import Sequential
from keras.layers.recurrent import LSTM, GRU,SimpleRNN
from keras.layers.core import Dense, Activation, Dropout
from keras.layers.embeddings import Embedding
from keras.layers.normalization import BatchNormalization
from keras.utils import np_utils
from sklearn import preprocessing, decomposition, model_selection, metrics, pipeline
from keras.layers import GlobalMaxPooling1D, Conv1D, MaxPooling1D, Flatten, Bidirectional, SpatialDropout1D
from keras.preprocessing import sequence, text
from keras.callbacks import EarlyStopping


import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from plotly import graph_objs as go
import plotly.express as px
import plotly.figure_factory as ff

Using TensorFlow backend.
/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [3]:
!pip install transformers
import os
import tensorflow as tf
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint

import transformers

from tokenizers import BertWordPieceTokenizer

     |████████████████████████████████| 645kB 2.7MB/s 
     |████████████████████████████████| 1.0MB 13.8MB/s 
     |████████████████████████████████| 3.8MB 19.1MB/s 
     |████████████████████████████████| 890kB 30.3MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893260 sha256=a2c628317cb30411a5038ef3ff90743f2562d34f68b97b05fa2a4bae51a4ca61
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [0]:
train1 = pd.read_csv("/content/train_2kmZucJ.csv")
# valid = pd.read_csv('/kaggle/input/jigsaw-multilingual-toxic-comment-classification/validation.csv')
test = pd.read_csv('/content/test_oJQbWVk.csv')


In [0]:
X = train1.drop('label',axis=1)
y = train1['label']

In [0]:
from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.33, random_state=42)

In [0]:
def fast_encode(texts, tokenizer, chunk_size=256, maxlen=512):
    """
    Encoder for encoding the text into sequence of integers for BERT Input
    """
    tokenizer.enable_truncation(max_length=maxlen)
    tokenizer.enable_padding(max_length=maxlen)
    all_ids = []
    
    for i in tqdm(range(0, len(texts), chunk_size)):
        text_chunk = texts[i:i+chunk_size].tolist()
        encs = tokenizer.encode_batch(text_chunk)
        all_ids.extend([enc.ids for enc in encs])
    
    return np.array(all_ids)

In [0]:
#IMP DATA FOR CONFIG

AUTO = tf.data.experimental.AUTOTUNE


# Configuration
EPOCHS = 3
BATCH_SIZE = 32
MAX_LEN = 192

In [9]:
# First load the real tokenizer
tokenizer = transformers.DistilBertTokenizer.from_pretrained('distilbert-base-multilingual-cased')
# Save the loaded tokenizer locally
tokenizer.save_pretrained('.')
# Reload it with the huggingface tokenizers library
fast_tokenizer = BertWordPieceTokenizer('vocab.txt', lowercase=False)
fast_tokenizer

Tokenizer(vocabulary_size=119547, model=BertWordPiece, unk_token=[UNK], sep_token=[SEP], cls_token=[CLS], pad_token=[PAD], mask_token=[MASK], clean_text=True, handle_chinese_chars=True, strip_accents=True, lowercase=False, wordpieces_prefix=##)

In [10]:
x_train = fast_encode(X_train.tweet.astype(str), fast_tokenizer, maxlen=MAX_LEN)
x_valid = fast_encode(X_valid.tweet.astype(str), fast_tokenizer, maxlen=MAX_LEN)
x_test = fast_encode(test.tweet.astype(str), fast_tokenizer, maxlen=MAX_LEN)

y_train = y_train.values
y_valid = y_valid.values

100%|██████████| 8/8 [00:00<00:00, 53.05it/s]


In [0]:
train_dataset = (
    tf.data.Dataset
    .from_tensor_slices((x_train, y_train))
    .repeat()
    .shuffle(2048)
    .batch(BATCH_SIZE)
    .prefetch(AUTO)
)
valid_dataset = (
    tf.data.Dataset
    .from_tensor_slices((x_valid, y_valid))
    .batch(BATCH_SIZE)
    .cache()
    .prefetch(AUTO)
)

test_dataset = (
    tf.data.Dataset
    .from_tensor_slices(x_test)
    .batch(BATCH_SIZE)
)

In [0]:
def build_model(transformer, max_len=512):
    """
    function for training the BERT model
    """
    input_word_ids = Input(shape=(max_len,), dtype=tf.int32, name="input_word_ids")
    sequence_output = transformer(input_word_ids)[0]
    cls_token = sequence_output[:, 0, :]
    out = Dense(1, activation='sigmoid')(cls_token)
    
    model = Model(inputs=input_word_ids, outputs=out)
    model.compile(Adam(lr=1e-5), loss='binary_crossentropy', metrics=['accuracy'])
    
    return model

In [13]:
%%time

transformer_layer = (transformers.TFDistilBertModel.from_pretrained('distilbert-base-multilingual-cased'))
model = build_model(transformer_layer, max_len=MAX_LEN)
model.summary()


Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_word_ids (InputLayer)  [(None, 192)]             0         
_________________________________________________________________
tf_distil_bert_model (TFDist ((None, 192, 768),)       134734080 
_________________________________________________________________
tf_op_layer_strided_slice (T [(None, 768)]             0         
_________________________________________________________________
dense (Dense)                (None, 1)                 769       
Total params: 134,734,849
Trainable params: 134,734,849
Non-trainable params: 0
_________________________________________________________________
CPU times: user 20.1 s, sys: 4.86 s, total: 24.9 s
Wall time: 25.3 s


In [15]:
n_steps = x_train.shape[0] // BATCH_SIZE
train_history = model.fit(
    train_dataset,
    steps_per_epoch=n_steps,
    validation_data=valid_dataset,
    epochs=EPOCHS
)

Epoch 1/3
165/165 [==============================] - 5921s 36s/step - loss: 0.3240 - accuracy: 0.8606 - val_loss: 0.3165 - val_accuracy: 0.8642
Epoch 2/3
165/165 [==============================] - 5889s 36s/step - loss: 0.2639 - accuracy: 0.8801 - val_loss: 0.2510 - val_accuracy: 0.8891
Epoch 3/3
165/165 [==============================] - 5912s 36s/step - loss: 0.2251 - accuracy: 0.9059 - val_loss: 0.2334 - val_accuracy: 0.9009


In [16]:
n_steps = x_valid.shape[0] // BATCH_SIZE
train_history_2 = model.fit(
    valid_dataset.repeat(),
    steps_per_epoch=n_steps,
    epochs=5
)

Epoch 1/5
81/81 [==============================] - 2507s 31s/step - loss: 0.2507 - accuracy: 0.8924
Epoch 2/5
81/81 [==============================] - 2516s 31s/step - loss: 0.2222 - accuracy: 0.9063
Epoch 3/5
81/81 [==============================] - 2504s 31s/step - loss: 0.1901 - accuracy: 0.9160
Epoch 4/5
81/81 [==============================] - 2505s 31s/step - loss: 0.1626 - accuracy: 0.9365
Epoch 5/5
81/81 [==============================] - 2503s 31s/step - loss: 0.1470 - accuracy: 0.9419


In [17]:
pred = model.predict(test_dataset, verbose=1)
pred
# sub.to_csv('submission.csv', index=False)

62/62 [==============================] - 573s 9s/step


array([[0.94613135],
       [0.6546349 ],
       [0.7700554 ],
       ...,
       [0.03686678],
       [0.05180049],
       [0.00165084]], dtype=float32)

In [21]:
pred = pred.astype(int)
pred

array([[0],
       [0],
       [0],
       ...,
       [0],
       [0],
       [0]])

In [24]:
test.head()

,id,tweet,label
0,7921,I hate the new #iphone upgrade. Won't let me d...,0.946131
1,7922,currently shitting my fucking pants. #apple #i...,0.654635
2,7923,"I'd like to puts some CD-ROMS on my iPad, is t...",0.770055
3,7924,My ipod is officially dead. I lost all my pict...,0.016772
4,7925,Been fighting iTunes all night! I only want th...,0.242467


In [0]:
test['label'] = model.predict(test_dataset, verbose=1)
test.to_csv('submission.csv', index=False)

